In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [2]:
import dgl
import numpy as np
import torch

from src.data import load_data

In [3]:
A, b, c, resultados, objetivos = load_data("../data/raw/97_9.jl")

len(A), len(b), resultados.shape, objetivos.shape

Restricted license - for non-production use only - expires 2024-10-28


(9, 9, (9, 1000, 194), (9, 1000))

In [4]:
optimals = torch.from_numpy(np.array([resultados[i,j] for i,j in enumerate(objetivos.argmax(-1))]))
optimals.shape

torch.Size([9, 194])

In [5]:
opt = optimals[5]

opt.shape

torch.Size([194])

In [6]:
torch.randint

<function torch._VariableFunctionsClass.randint>

In [7]:
candidate = torch.randint(0,2,opt.shape)
candidate_label = (candidate == opt).type(candidate.type())

candidate, candidate_label

(tensor([1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
         0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
         1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
         1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
         1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
         1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
         1, 1]),
 tensor([0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
         1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
         1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
         0,

In [16]:
from src.dataset import VarClassDataset


ds = VarClassDataset(A, b, c, optimals, samples_per_problem=2)

g, y = ds[5]
assert all((g.nodes['var'].data['x'] == optimals[2]).type(opt.type()) == y)

In [17]:
from src.net import GCN

net = GCN(2,1, readout_op=None)

y_hat = net(g)
y_hat.shape

torch.Size([1, 194])

In [18]:
dl = dgl.dataloading.GraphDataLoader(
    ds,
    batch_size=3,
    drop_last=False
)
g, y = next(iter(dl))
y.shape

torch.Size([3, 194])

In [19]:
y_hat = net(g)
y_hat.shape

torch.Size([3, 194])

In [24]:
y.type()

'torch.LongTensor'

In [29]:
torch.nn.functional.binary_cross_entropy_with_logits(y_hat, y.double(), reduce=False)

tensor([[0.6831, 0.7028, 0.7023, 0.7017, 0.6845, 0.6847, 0.7013, 0.6850, 0.7011,
         0.7008, 0.6854, 0.7007, 0.6856, 0.7005, 0.7004, 0.7005, 0.7007, 0.7007,
         0.6856, 0.6856, 0.7008, 0.7008, 0.6856, 0.7008, 0.7009, 0.7008, 0.6856,
         0.7007, 0.6855, 0.7008, 0.7010, 0.6854, 0.7009, 0.6854, 0.6854, 0.7011,
         0.7011, 0.6854, 0.6854, 0.6854, 0.6853, 0.7011, 0.7011, 0.6853, 0.6852,
         0.6852, 0.7010, 0.7009, 0.6853, 0.6853, 0.7010, 0.7011, 0.7011, 0.7011,
         0.6853, 0.6853, 0.6853, 0.7010, 0.7009, 0.7010, 0.6853, 0.7011, 0.6852,
         0.7011, 0.6851, 0.6850, 0.7013, 0.7012, 0.6853, 0.7011, 0.6853, 0.7011,
         0.6853, 0.7011, 0.6854, 0.6855, 0.7011, 0.7010, 0.6853, 0.6853, 0.7009,
         0.7011, 0.6852, 0.7011, 0.6852, 0.6850, 0.6851, 0.7013, 0.7014, 0.7013,
         0.6847, 0.6846, 0.7019, 0.6844, 0.7023, 0.6837, 0.6835, 0.6845, 0.7032,
         0.7053, 0.6806, 0.6805, 0.6804, 0.6804, 0.7061, 0.7060, 0.6805, 0.7057,
         0.6807, 0.6809, 0.6